In [2]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [3]:
data = pd.read_csv('diamonds.txt', sep = '\t')
data.head()
X = data[["carat", "depth", "table", "x", "y", "z"]]
y = data["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [4]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

rand_for = RandomForestRegressor(random_state=1)
rand_for.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

In [5]:
lin_reg_predictions = abs(y_test-lin_reg.predict(X_test))
rand_for_predictions = abs(y_test-rand_for.predict(X_test))

In [6]:
scipy.stats.ttest_ind(lin_reg_predictions, rand_for_predictions)

Ttest_indResult(statistic=6.153129752743644, pvalue=7.703732222711213e-10)

In [7]:
cm = CompareMeans(DescrStatsW(lin_reg_predictions), DescrStatsW(rand_for_predictions))

In [8]:
cm.tconfint_diff(usevar='unequal')

(59.12439535320725, 114.39972888765313)

In [9]:
T = (9.5-9.57)/(0.4/np.sqrt(160))

print 2 * (1.0-T)

6.42718872423575


In [10]:
2.75/0.994426085089858

2.765414183349289

In [11]:
scipy.stats.norm.cdf(2.75)

0.9970202367649454

In [12]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

In [13]:
conf_interval_banner_a = proportion_confint(10, 
                                            34,
                                            method = 'wilson')
conf_interval_banner_b = proportion_confint(3, 
                                            16,
                                            method = 'wilson')
print conf_interval_banner_a
print conf_interval_banner_b

(0.16834630670422424, 0.4616890979471444)
(0.06591599071428139, 0.43008880961974144)


In [14]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [15]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [16]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [17]:
print "p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind([1] * 10 + [0] * 24, [1] * 4 + [0] * 12), 'greater')

p-value: 0.372930


In [18]:
data = pd.read_csv('banknotes.txt', sep = '\t')
data.head()
X = data[["X1", "X2", "X3", "X4", "X5", "X6"]]
y = data["real"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=50, random_state=1)
print len(X_test)

50


In [19]:
from sklearn.linear_model import LogisticRegression

In [20]:
reg_1 = LogisticRegression()
reg_1.fit(X_train[["X1", "X2", "X3"]], y_train)

reg_2 = LogisticRegression()
reg_2.fit(X_train[["X4", "X5", "X6"]], y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [21]:
predict1 = reg_1.predict(X_test[["X1", "X2", "X3"]])
predict2 = reg_2.predict(X_test[["X4", "X5", "X6"]])

In [22]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_rel(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [23]:
proportions_diff_z_test(proportions_diff_z_stat_rel(y_test != predict1, y_test != predict2))

0.0032969384555543435

In [24]:
proportions_diff_confint_rel(y_test != predict1, y_test != predict2)

(0.059945206279614305, 0.3000547937203857)

Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [25]:
z = (541.4- 525) / (100/(100**0.5))
print 1 - scipy.stats.norm.cdf(z)

0.05050258347410397


Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки

In [26]:
z = (541.5- 525) / (100/(100**0.5))
print 1 - scipy.stats.norm.cdf(z)

0.0494714680336481


Давайте вернёмся к данным выживаемости пациентов с лейкоцитарной лимфомой из видео про критерий знаков:


49,58,75,110,112,132,151,276,281,362 
∗
 

Измерено остаточное время жизни с момента начала наблюдения (в неделях); звёздочка обозначает цензурирование сверху — исследование длилось 7 лет, и остаточное время жизни одного пациента, который дожил до конца наблюдения, неизвестно.

Поскольку цензурировано только одно наблюдение, для проверки гипотезы 
medX=200 на этих данных можно использовать критерий знаковых рангов — можно считать, что время дожития последнего пациента в точности равно 362, на ранг этого наблюдения это никак не повлияет.

Критерием знаковых рангов проверьте эту гипотезу против двусторонней альтернативы, введите достигаемый уровень значимости, округлённый до четырёх знаков после десятичной точки.

In [27]:
a = np.array([49,58,75,110,112,132,151,276,281,362])
med = 200

stats.wilcoxon(a - med)

WilcoxonResult(statistic=17.0, pvalue=0.2845026979112075)

В ходе исследования влияния лесозаготовки на биоразнообразие лесов острова Борнео собраны данные о количестве видов деревьев в 12 лесах, где вырубка не ведётся:

22,22,15,13,19,19,18,20,21,13,13,15,

и в 9 лесах, где идёт вырубка:

17,18,18,15,12,4,14,15,10.

Проверьте гипотезу о равенстве среднего количества видов в двух типах лесов против односторонней альтернативы о снижении биоразнообразия в вырубаемых лесах. Используйте ранговый критерий. Чему равен достигаемый уровень значимости? Округлите до четырёх знаков после десятичной точки.

In [28]:
forest_1 = [22,22,15,13,19,19,18,20,21,13,13,15]
forest_2 = [17,18,18,15,12,4,14,15,10]

stats.mannwhitneyu(forest_1, forest_2)

MannwhitneyuResult(statistic=27.0, pvalue=0.02900499272087373)

С помощью бутстрепа постройте 95% доверительный интервал для разности средних температур воздуха при запусках, когда уплотнительные кольца повреждались, и запусках, когда повреждений не было. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после запятой.

Чтобы получить в точности такой же доверительный интервал, как у нас:

установите random seed = 0 перед первым вызовом функции get_bootstrap_samples, один раз
сделайте по 1000 псевдовыборок из каждой выборки.

In [29]:
challenger = pd.read_csv('challenger.txt', sep = '\t')
challenger.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [30]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [48]:
np.random.seed(0)
n_samples = 1000
challenger_1 = map(np.mean, get_bootstrap_samples(challenger[challenger.Incident == 1].Temperature.values, n_samples))
challenger_0 = map(np.mean, get_bootstrap_samples(challenger[challenger.Incident == 0].Temperature.values, n_samples))

In [32]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

delta_median_scores = map(lambda x: x[1] - x[0], zip(challenger_1, challenger_0))
print stat_intervals(delta_median_scores, 0.05)

[1.45040179 8.06457589]


In [33]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

def get_random_combinations(n1, n2, max_combinations):
    index = range(n1 + n2)
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [34]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [53]:
np.random.seed(0)
print "p-value: %f" % permutation_test(challenger[challenger.Incident == 1].Temperature.values, 
                                       challenger[challenger.Incident == 0].Temperature.values, 
                                       max_permutations = 10000)

p-value: 0.005700
